In [2]:
import pandas as pd
import os 

# Check Inference Data Preprocessing

In [6]:
city = "Bogor"
time_series_model = "ARIMA"
lst_time_series_feature = [
    'median_length', 
    'median_delay', 
    'median_speed_kmh', 
    'median_regular_speed', 
    'median_delay_seconds'
]
dataset_path_template = "../data/processed/final_dataset_{}.parquet.gzip"
time_series_result_template = "../data/model_result/time_series/{}_{}_{}.parquet.gzip"

In [11]:
df = pd.read_parquet(dataset_path_template.format(city))
df = df[df['classification_split'] == 'test']
df

,time,street,level,median_length,median_delay,median_speed_kmh,median_regular_speed,median_delay_seconds,rain_intensity,holiday_gap,time_series_split,classification_split
61410,2022-08-31 00:00:00,Surya Kencana,0,1192.140229,10.148264,21.323593,20.414154,12.697850,0.0,-1,test,test
61411,2022-08-31 00:00:00,RE Martadinata,0,1221.311475,11.311475,27.311475,21.088608,11.129032,0.0,-1,test,test
61412,2022-08-31 00:00:00,Binamarga,0,1187.818389,9.843616,25.889055,22.174524,10.143151,0.0,-1,test,test
61413,2022-08-31 00:00:00,Jenderal Sudirman,0,1578.128999,9.626991,31.258333,22.820000,78.500000,0.0,-1,test,test
61414,2022-08-31 00:00:00,Taman Cimanggu Raya,0,1338.483162,9.893030,24.729622,22.604954,11.497557,0.0,-1,test,test
...,...,...,...,...,...,...,...,...,...,...,...,...
66833,2022-09-04 21:00:00,Cilebut Raya,0,1439.817561,11.573804,23.706937,19.270000,60.500000,0.0,-1,test,test
66834,2022-09-04 21:00:00,Semplak Raya,3,539.000000,152.000000,8.610000,19.470000,79.000000,0.0,-1,test,test
66835,2022-09-04 21:00:00,Pahlawan,0,1799.920448,9.599060,32.400000,19.705000,74.000000,0.0,-1,test,test
66836,2022-09-04 21:00:00,N9 KH Soleh Iskandar,3,442.000000,108.000000,11.540000,23.987499,76.000000,0.0,-1,test,test


In [42]:
time_series_pred = {}
for feature in lst_time_series_feature:
    time_series_pred[feature] = pd.read_parquet(
        time_series_result_template.format(time_series_model, city, feature)
    )

In [32]:
time_series_pred['median_delay']

,street,time,actual,pred
0,Kebon Pedes,2022-08-24 00:00:00,9.937105,25.285189
1,Kebon Pedes,2022-08-24 01:00:00,8.997109,28.294072
2,Kebon Pedes,2022-08-24 02:00:00,10.242862,30.332896
3,Kebon Pedes,2022-08-24 03:00:00,10.738484,31.714406
4,Kebon Pedes,2022-08-24 04:00:00,9.511229,32.650520
...,...,...,...,...
13151,N9 Ir Haji Juanda,2022-09-04 17:00:00,125.000000,68.196574
13152,N9 Ir Haji Juanda,2022-09-04 18:00:00,207.000000,68.196574
13153,N9 Ir Haji Juanda,2022-09-04 19:00:00,89.000000,68.196574
13154,N9 Ir Haji Juanda,2022-09-04 20:00:00,9.056381,68.196574


In [23]:
tmp = df.drop(['median_delay'], axis=1)
ts = time_series_pred['median_delay']
ts.drop(['actual'], axis=1, inplace=True)
ts.rename(columns={
    'pred': 'median_delay'
}, inplace=True)
tmp.join(
    ts.set_index(['time', 'street']),
    on=['time', 'street'],
    how='inner'
)

,time,street,level,median_length,median_speed_kmh,median_regular_speed,median_delay_seconds,rain_intensity,holiday_gap,time_series_split,classification_split,median_delay
61410,2022-08-31 00:00:00,Surya Kencana,0,1192.140229,21.323593,20.414154,12.697850,0.0,-1,test,test,119.705286
61411,2022-08-31 00:00:00,RE Martadinata,0,1221.311475,27.311475,21.088608,11.129032,0.0,-1,test,test,56.097950
61412,2022-08-31 00:00:00,Binamarga,0,1187.818389,25.889055,22.174524,10.143151,0.0,-1,test,test,46.039784
61413,2022-08-31 00:00:00,Jenderal Sudirman,0,1578.128999,31.258333,22.820000,78.500000,0.0,-1,test,test,44.320485
61414,2022-08-31 00:00:00,Taman Cimanggu Raya,0,1338.483162,24.729622,22.604954,11.497557,0.0,-1,test,test,54.992715
...,...,...,...,...,...,...,...,...,...,...,...,...
66833,2022-09-04 21:00:00,Cilebut Raya,0,1439.817561,23.706937,19.270000,60.500000,0.0,-1,test,test,55.620497
66834,2022-09-04 21:00:00,Semplak Raya,3,539.000000,8.610000,19.470000,79.000000,0.0,-1,test,test,99.054059
66835,2022-09-04 21:00:00,Pahlawan,0,1799.920448,32.400000,19.705000,74.000000,0.0,-1,test,test,75.917544
66836,2022-09-04 21:00:00,N9 KH Soleh Iskandar,3,442.000000,11.540000,23.987499,76.000000,0.0,-1,test,test,88.606043


In [28]:
def change_feature(dataset, time_series):
    result = dataset.copy()
    result.drop(lst_time_series_feature, axis=1, inplace=True)
    for feature in lst_time_series_feature:
        data = time_series[feature]
        data.drop(['actual'], axis=1, inplace=True)
        data.rename(columns={
            'pred': feature
        }, inplace=True)
        result = result.join(
            data.set_index(['time', 'street']),
            on=['time', 'street'],
            how='inner',
        )
    return result

In [43]:
a = change_feature(df, time_series_pred)
a.replace(-1, 8, inplace=True)
a

,time,street,level,rain_intensity,holiday_gap,time_series_split,classification_split,median_length,median_delay,median_speed_kmh,median_regular_speed,median_delay_seconds
61410,2022-08-31 00:00:00,Surya Kencana,0,0.0,8,test,test,1083.263515,119.705286,24.143504,18.360732,45.344944
61411,2022-08-31 00:00:00,RE Martadinata,0,0.0,8,test,test,947.237458,56.097950,19.098030,21.914765,11.568716
61412,2022-08-31 00:00:00,Binamarga,0,0.0,8,test,test,943.254476,46.039784,21.045616,21.906465,12.135207
61413,2022-08-31 00:00:00,Jenderal Sudirman,0,0.0,8,test,test,1315.302525,44.320485,30.498454,22.302701,79.963886
61414,2022-08-31 00:00:00,Taman Cimanggu Raya,0,0.0,8,test,test,1118.102850,54.992715,20.371296,22.729501,11.347078
...,...,...,...,...,...,...,...,...,...,...,...,...
66833,2022-09-04 21:00:00,Cilebut Raya,0,0.0,8,test,test,1207.883194,55.620497,19.964025,19.613838,59.001235
66834,2022-09-04 21:00:00,Semplak Raya,3,0.0,8,test,test,1293.624576,99.054059,20.261813,20.637772,76.863704
66835,2022-09-04 21:00:00,Pahlawan,0,0.0,8,test,test,1264.702578,75.917544,23.475143,19.840296,71.433882
66836,2022-09-04 21:00:00,N9 KH Soleh Iskandar,3,0.0,8,test,test,1414.831178,88.606043,31.903925,23.681871,81.046347


In [38]:
tmp = time_series_pred['median_length']
tmp[
    (tmp['street'] == 'Binamarga') & \
        (tmp['time'] == '2022-08-31')
]

,street,time,median_length
8462,Binamarga,2022-08-31,943.254476


# Result Analysis

In [7]:
def change_format(data):
    tmp = str(data)
    return tmp.replace('.', ',')

## Baseline

In [8]:
lst_file = os.listdir("../data/eval_result/classifier/")
path_template = "../data/eval_result/classifier/{}"

result_df = pd.DataFrame(columns=['model_conf', 'accuracy', 'precision', 'recall', 'f1-score'])

used_file = [filename for filename in lst_file if 'baseline' in filename]

for filename in used_file:
    data = pd.read_csv(path_template.format(filename))
    for col in data.columns:
        data[col] = data[col].apply(change_format)
    result_df = result_df.append(
        data,
        ignore_index=True
    )

result_df

C:\Users\Gibran Brahmanta\AppData\Local\Temp\ipykernel_3232\4282022862.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(
C:\Users\Gibran Brahmanta\AppData\Local\Temp\ipykernel_3232\4282022862.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(
C:\Users\Gibran Brahmanta\AppData\Local\Temp\ipykernel_3232\4282022862.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(
C:\Users\Gibran Brahmanta\AppData\Local\Temp\ipykernel_3232\4282022862.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(
C:\Users\Gibran Brahmanta\Ap

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_DecisionTree_baseline,"0,5980103168754606","0,7239645644549594","0,5246202350566868","0,5693710031838252"
1,Bogor_LightGBM_baseline,"0,6099852616064849","0,7316758242904986","0,5349905703603695","0,5820979144086537"
2,Bogor_LogisticRegression_baseline,"0,5987472365512159","0,7237157287157286","0,5311137089468679","0,5804832343568881"
3,Bogor_NaiveBayes_baseline,"0,5029476787030214","0,2616290018832392","0,4935127820694275","0,3405753376407235"
4,Bogor_RandomForest_baseline,"0,6051952837140752","0,7280823942886622","0,5315620046699833","0,5786113255185604"
5,Bogor_SVM_baseline,"0,5712969786293294","0,7082211050359009","0,5001859663274294","0,5312323206816746"
6,Bogor_XGBoost_baseline,"0,6068533529845247","0,7290624573335697","0,5338354019410458","0,5818456257866101"


## Hyperparameter Tuning

In [21]:
city = 'Bogor'

def get_hpt_result(city, model_name, partition):
    path_template = "../data/eval_result/classifier/{}"
    filename = "{}_{}_HyperparameterTuning_{}.csv".format(
        city,
        model_name,
        partition
    )
    data = pd.read_csv(path_template.format(filename))
    data.drop(columns=['Unnamed: 0'], inplace=True)
    for col in data.columns:
        data[col] = data[col].apply(change_format)
    if partition == 'valid':
        return data.iloc[0:10,:]
    else:
        return data

### Logistic Regression

In [22]:
get_hpt_result(city, 'LogisticRegression', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_LogisticRegression_(penalty_l2)(C_1,0)(s...","0,6187888198757764","0,7355276917800458","0,5399944911172169","0,5886348361242816"
1,"Bogor_LogisticRegression_(penalty_l2)(C_0,1)(s...","0,6200828157349897","0,7357655941999933","0,5393234163684947","0,5878108903535167"
2,"Bogor_LogisticRegression_(penalty_l2)(C_0,01)(...","0,6158126293995859","0,7341114969627459","0,5390443378920559","0,587661160610404"
3,"Bogor_LogisticRegression_(penalty_l1)(C_1,0)(s...","0,6033902691511387","0,7256777630611538","0,5303386923289698","0,5791047880618878"
4,"Bogor_LogisticRegression_(penalty_l1)(C_0,1)(s...","0,6033902691511387","0,7256777630611538","0,5303386923289698","0,5791047880618878"
5,"Bogor_LogisticRegression_(penalty_l2)(C_1,0)(s...","0,6033902691511387","0,7256777630611538","0,5303386923289698","0,5791047880618878"
6,"Bogor_LogisticRegression_(penalty_l2)(C_0,1)(s...","0,6033902691511387","0,7256777630611538","0,5303386923289698","0,5791047880618878"
7,"Bogor_LogisticRegression_(penalty_l2)(C_0,01)(...","0,6033902691511387","0,7256777630611538","0,5303386923289698","0,5791047880618878"
8,"Bogor_LogisticRegression_(penalty_l1)(C_0,01)(...","0,6031314699792961","0,7255125831454884","0,5301733484136258","0,5789408944325074"
9,"Bogor_LogisticRegression_(penalty_l2)(C_0,01)(...","0,6130952380952381","0,7303816653997158","0,5299204574730226","0,576592102148768"


In [23]:
get_hpt_result(city, 'LogisticRegression', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_LogisticRegression_(penalty_l1)(C_1,0)(s...","0,606484893146647","0,7292405139019756","0,5369666579078619","0,5862897218472081"
1,"Bogor_LogisticRegression_(penalty_l1)(C_0,1)(s...","0,606484893146647","0,7292405139019756","0,5369666579078619","0,5862897218472081"
2,"Bogor_LogisticRegression_(penalty_l2)(C_1,0)(s...","0,606484893146647","0,7292405139019756","0,5369666579078619","0,5862897218472081"
3,"Bogor_LogisticRegression_(penalty_l2)(C_0,1)(s...","0,606484893146647","0,7292405139019756","0,5369666579078619","0,5862897218472081"
4,"Bogor_LogisticRegression_(penalty_l2)(C_0,01)(...","0,606484893146647","0,7292405139019756","0,5369666579078619","0,5862897218472081"
5,"Bogor_LogisticRegression_(penalty_l1)(C_0,01)(...","0,606484893146647","0,7292405139019756","0,5369666579078619","0,5862897218472081"
6,"Bogor_LogisticRegression_(penalty_l2)(C_0,01)(...","0,5983787767133383","0,7236489423355549","0,5312399490224972","0,5806503500955196"
7,"Bogor_LogisticRegression_(penalty_l2)(C_1,0)(s...","0,5987472365512159","0,7237157287157286","0,5311137089468679","0,5804832343568881"
8,"Bogor_LogisticRegression_(penalty_l2)(C_0,1)(s...","0,5952468680913781","0,7210768443214103","0,5280814366757157","0,5773786613128692"
9,"Bogor_LogisticRegression_(penalty_l2)(C_0,01)(...","0,5946941783345615","0,7208223509463837","0,5229966872658522","0,5687026015370572"


### SVM

In [24]:
get_hpt_result(city, 'SVM', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_SVM_(C_0,01)(multi_class_crammer_singer)...","0,6230590062111802","0,7395862891602287","0,5449491167446967","0,5933742826431921"
1,"Bogor_SVM_(C_0,01)(multi_class_crammer_singer)...","0,6233178053830227","0,7395254385174496","0,5447298918467516","0,5932250879631499"
2,"Bogor_SVM_(C_0,1)(multi_class_crammer_singer)(...","0,6152950310559007","0,7350498909582758","0,5404948108808123","0,5887825921294421"
3,"Bogor_SVM_(C_0,1)(multi_class_crammer_singer)(...","0,6140010351966874","0,7317827012871323","0,5356604794603431","0,5842924591531188"
4,"Bogor_SVM_(C_0,01)(multi_class_ovr)(loss_hinge)","0,6177536231884058","0,7340468091559681","0,5345766064493708","0,5820974352369942"
5,"Bogor_SVM_(C_1,0)(multi_class_crammer_singer)(...","0,6004140786749482","0,7285675516491863","0,5335580746584169","0,5804751034991772"
6,"Bogor_SVM_(C_0,01)(multi_class_ovr)(loss_squar...","0,6096014492753623","0,7280690867982241","0,5286598568810311","0,5760986898051197"
7,"Bogor_SVM_(C_1,0)(multi_class_crammer_singer)(...","0,59277950310559","0,7154166833509074","0,5166171126968081","0,5639383726454766"
8,"Bogor_SVM_(C_0,1)(multi_class_ovr)(loss_square...","0,5900621118012422","0,7182850060288873","0,5053275026450416","0,5409444503805082"
9,"Bogor_SVM_(C_1,0)(multi_class_ovr)(loss_square...","0,5750517598343685","0,7075575260566037","0,4904750349521909","0,5147275248514861"


In [26]:
get_hpt_result(city, 'SVM', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_SVM_(C_1,0)(multi_class_crammer_singer)(...","0,599852616064849","0,7282084288367602","0,5362544056629762","0,5844029639633822"
1,"Bogor_SVM_(C_0,1)(multi_class_crammer_singer)(...","0,6016949152542372","0,7267787469030912","0,5347850716008934","0,5841112967149431"
2,"Bogor_SVM_(C_0,1)(multi_class_crammer_singer)(...","0,6004053058216654","0,7246158702532995","0,5312406662956543","0,5803585783893603"
3,"Bogor_SVM_(C_0,01)(multi_class_crammer_singer)...","0,5981945467943994","0,723289673609909","0,5305467045162716","0,5798998062360216"
4,"Bogor_SVM_(C_0,01)(multi_class_ovr)(loss_hinge)","0,6040899042004422","0,7275960169213253","0,5303257843839204","0,5769094331556143"
5,"Bogor_SVM_(C_0,01)(multi_class_crammer_singer)...","0,5913780397936624","0,7189400354091294","0,5264019415786364","0,5759049547044258"
6,"Bogor_SVM_(C_0,01)(multi_class_ovr)(loss_squar...","0,6000368459837878","0,7244126152353226","0,5277486219308749","0,574623977120556"
7,"Bogor_SVM_(C_1,0)(multi_class_crammer_singer)(...","0,5838246131171703","0,7126424087178093","0,5163360887300937","0,5632451295149908"
8,"Bogor_SVM_(C_0,1)(multi_class_ovr)(loss_square...","0,5766396462785557","0,7126566935174531","0,5043655170131784","0,536975902942991"
9,"Bogor_SVM_(C_1,0)(multi_class_ovr)(loss_square...","0,5648489314664702","0,7064666996443661","0,4934618891172777","0,5158736720074534"


### Naive Bayes

In [27]:
get_hpt_result(city, 'NaiveBayes', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_NaiveBayes_(alpha_1,0)(norm_True)","0,535455486542443","0,329310253733709","0,4753214256770055","0,3815975128550075"
1,"Bogor_NaiveBayes_(alpha_0,1)(norm_True)","0,535455486542443","0,329310253733709","0,4753214256770055","0,3815975128550075"
2,"Bogor_NaiveBayes_(alpha_0,01)(norm_True)","0,535455486542443","0,329310253733709","0,4753214256770055","0,3815975128550075"
3,"Bogor_NaiveBayes_(alpha_1,0)(norm_False)","0,5182453416149069","0,51990374317824","0,4947827410041823","0,3484582884797831"
4,"Bogor_NaiveBayes_(alpha_0,1)(norm_False)","0,5182453416149069","0,51990374317824","0,4947827410041823","0,3484582884797831"
5,"Bogor_NaiveBayes_(alpha_0,01)(norm_False)","0,5182453416149069","0,51990374317824","0,4947827410041823","0,3484582884797831"


In [28]:
get_hpt_result(city, 'NaiveBayes', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_NaiveBayes_(alpha_1,0)(norm_True)","0,5180545320560059","0,3089233051555746","0,4739438694177787","0,3690795931105892"
1,"Bogor_NaiveBayes_(alpha_0,1)(norm_True)","0,5180545320560059","0,3089233051555746","0,4739438694177787","0,3690795931105892"
2,"Bogor_NaiveBayes_(alpha_0,01)(norm_True)","0,5180545320560059","0,3089233051555746","0,4739438694177787","0,3690795931105892"
3,"Bogor_NaiveBayes_(alpha_1,0)(norm_False)","0,5029476787030214","0,2616290018832392","0,4935127820694275","0,3405753376407235"
4,"Bogor_NaiveBayes_(alpha_0,1)(norm_False)","0,5029476787030214","0,2616290018832392","0,4935127820694275","0,3405753376407235"
5,"Bogor_NaiveBayes_(alpha_0,01)(norm_False)","0,5029476787030214","0,2616290018832392","0,4935127820694275","0,3405753376407235"


### Decision Tree

In [29]:
get_hpt_result(city, 'DecisionTree', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6347049689440993","0,7463461719530677","0,5499202563445797","0,5983781600996189"
1,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6414337474120083","0,7528411296344825","0,5498472579503403","0,5967182711508984"
2,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6233178053830227","0,740132453938204","0,545640712720331","0,5939792671637605"
3,Bogor_DecisionTree_(criterion_log_loss)(splitt...,"0,6306935817805382","0,7439152792938585","0,5438760658732898","0,5915258175638058"
4,Bogor_DecisionTree_(criterion_log_loss)(splitt...,"0,625","0,7394234209156982","0,5426471149347448","0,591114152105324"
5,Bogor_DecisionTree_(criterion_gini)(splitter_r...,"0,6163302277432712","0,7374858379400884","0,5432611947833493","0,5907817067397599"
6,Bogor_DecisionTree_(criterion_gini)(splitter_b...,"0,6213768115942029","0,7284580108033399","0,5427004343912366","0,590012097556455"
7,Bogor_DecisionTree_(criterion_gini)(splitter_b...,"0,6222826086956522","0,7376147883124627","0,5409312442874911","0,5894600516136959"
8,Bogor_DecisionTree_(criterion_log_loss)(splitt...,"0,6234472049689441","0,7382141400650349","0,5409061542799606","0,5892531945656606"
9,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,629787784679089","0,7440019740293544","0,5412733157838541","0,5878287175105016"


In [30]:
get_hpt_result(city, 'DecisionTree', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6258290346352248","0,7424440896482509","0,5503316086419552","0,5994277019042403"
1,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6050110537951363","0,7298802372898031","0,5381462136145231","0,5871500647337767"
2,Bogor_DecisionTree_(criterion_log_loss)(splitt...,"0,6088798820928519","0,7303673068605354","0,5365958276857009","0,5853839848136927"
3,Bogor_DecisionTree_(criterion_gini)(splitter_b...,"0,6061164333087693","0,7287153908794789","0,5357437071752033","0,5849488194755726"
4,Bogor_DecisionTree_(criterion_log_loss)(splitt...,"0,6099852616064849","0,7311187139020712","0,5358900308992282","0,5837484718793425"
5,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6098010316875461","0,7314366414232111","0,5347879406935213","0,5818112699773108"
6,Bogor_DecisionTree_(criterion_log_loss)(splitt...,"0,6048268238761975","0,7273915342104162","0,5319744367352495","0,5796189754519646"
7,Bogor_DecisionTree_(criterion_entropy)(splitte...,"0,6083271923360354","0,73153806402492","0,5328193845141772","0,5788914252183841"
8,Bogor_DecisionTree_(criterion_gini)(splitter_r...,"0,5797715549005158","0,7157557607982364","0,5228557430905048","0,5706668874172595"
9,Bogor_DecisionTree_(criterion_gini)(splitter_b...,"0,5825350036845984","0,7123456534264808","0,5192310031916834","0,5686890171739362"


### Random Forest

In [31]:
get_hpt_result(city, 'RandomForest', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6380693581780539","0,7486106055090401","0,5507743375571821","0,5989373288667427"
1,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6376811594202898","0,7483298134565591","0,5505060812203089","0,5986717330786541"
2,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6366459627329193","0,7475624138011815","0,5499054269505053","0,598103907071673"
3,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6363871635610766","0,7474191699818117","0,549719842571304","0,597925497984924"
4,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6358695652173914","0,7470271060832082","0,5494296356683307","0,5976520738163339"
5,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6359989648033126","0,7471160303958949","0,5494111053067161","0,5976058281725476"
6,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6350931677018633","0,7464547706635493","0,5492372108801588","0,5975362504008511"
7,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,63574016563147","0,7469314847139478","0,5492862423190868","0,5974963616122145"
8,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6358695652173914","0,7470215841054063","0,5492879524213296","0,5974749775100929"
9,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6369047619047619","0,7479516717977233","0,5494230760224148","0,5974629250961864"


In [32]:
get_hpt_result(city, 'RandomForest', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6195652173913043","0,738166643812958","0,5429311428447671","0,590986515284402"
1,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6190125276344879","0,7377743001595115","0,5425072344089891","0,5905407538913267"
2,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,621039056742815","0,7398381292353226","0,5425488362520942","0,5894194321136329"
3,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6162490788504053","0,7357258019054062","0,5407556533596328","0,5889308307867183"
4,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6147752394988946","0,7346250602473355","0,5397274422890955","0,5879114074795325"
5,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6125644804716286","0,7330173682006662","0,5389517113698167","0,5876055722251078"
6,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6116433308769345","0,7323593213965703","0,5380816590303944","0,5866244803419606"
7,Bogor_RandomForest_(criterion_entropy)(n_estim...,"0,6121960206337509","0,7327674810603937","0,5381784909065873","0,5865483505614679"
8,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6109064112011791","0,731832976105422","0,5375164477826907","0,5860378435955361"
9,Bogor_RandomForest_(criterion_log_loss)(n_esti...,"0,6099852616064849","0,7311458087400027","0,5367690491531126","0,5852356187966975"


### LightGBM

In [33]:
get_hpt_result(city, 'LightGBM', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6420807453416149","0,7519804095998689","0,5520417785581443","0,5997107441205789"
1,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6420807453416149","0,7519804095998689","0,5520417785581443","0,5997107441205789"
2,"Bogor_LightGBM_(learning_rate_0,001)(n_estimat...","0,6427277432712215","0,7533047163373262","0,5508156607740611","0,597664541121787"
3,"Bogor_LightGBM_(learning_rate_0,001)(n_estimat...","0,6427277432712215","0,7533047163373262","0,5508156607740611","0,597664541121787"
4,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6394927536231884","0,7500369204942058","0,5500442515191307","0,5976279778835063"
5,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6317287784679089","0,7441922086634575","0,5453673005528156","0,5932756322406837"
6,"Bogor_LightGBM_(learning_rate_0,001)(n_estimat...","0,6406573498964804","0,7533280244812025","0,5475498249210069","0,5931646290518332"
7,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6321169772256728","0,7445475486488083","0,5449878620462546","0,5926314520670375"
8,"Bogor_LightGBM_(learning_rate_0,1)(n_estimator...","0,629399585921325","0,7425229441251772","0,543474396037574","0,5912916712168143"
9,"Bogor_LightGBM_(learning_rate_0,1)(n_estimator...","0,6292701863354038","0,7424550099011328","0,5433714836160447","0,5911878282481253"


In [34]:
get_hpt_result(city, 'LightGBM', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6212232866617539","0,7396674814303305","0,5436918110277491","0,5914668135599088"
1,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6212232866617539","0,7396674814303305","0,5436918110277491","0,5914668135599088"
2,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6169859985261607","0,7363751488365522","0,5409324611928296","0,5888721762725319"
3,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6147752394988946","0,7347823308782839","0,5396047885792511","0,5877219714806493"
4,"Bogor_LightGBM_(learning_rate_0,001)(n_estimat...","0,6190125276344879","0,7387343750024454","0,5409127361810124","0,5875992009140308"
5,"Bogor_LightGBM_(learning_rate_0,001)(n_estimat...","0,6190125276344879","0,7387343750024454","0,5409127361810124","0,5875992009140308"
6,"Bogor_LightGBM_(learning_rate_0,01)(n_estimato...","0,6153279292557111","0,7356613735578131","0,5385772947818708","0,585505856242929"
7,"Bogor_LightGBM_(learning_rate_0,1)(n_estimator...","0,6099852616064849","0,7316758242904986","0,5349905703603695","0,5820979144086537"
8,"Bogor_LightGBM_(learning_rate_0,001)(n_estimat...","0,6138540899042004","0,7362314478506148","0,5358932586284345","0,5809212418025859"
9,"Bogor_LightGBM_(learning_rate_0,1)(n_estimator...","0,6068533529845247","0,7292091121376202","0,5332425756767981","0,5807750292988678"


### XGBoost

In [35]:
get_hpt_result(city, 'XGBoost', 'valid')

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_XGBoost_(n_estimators_500)(max_depth_10)...,"0,6490683229813664","0,7578266373158632","0,5554535601518492","0,6024039327422327"
1,Bogor_XGBoost_(n_estimators_250)(max_depth_10)...,"0,6431159420289855","0,7526200274348422","0,5527031542195199","0,6003235909155991"
2,Bogor_XGBoost_(n_estimators_100)(max_depth_10)...,"0,6330227743271222","0,7451988524497947","0,5490884064611322","0,5975835307057883"
3,Bogor_XGBoost_(n_estimators_100)(max_depth_10)...,"0,6371635610766045","0,7486805029342203","0,548029904220745","0,5954768717424028"
4,Bogor_XGBoost_(n_estimators_250)(max_depth_50)...,"0,6374223602484472","0,749112604344209","0,5479118816420865","0,5952282405177182"
5,Bogor_XGBoost_(n_estimators_250)(max_depth_100...,"0,6374223602484472","0,749112604344209","0,5479118816420865","0,5952282405177182"
6,Bogor_XGBoost_(n_estimators_100)(max_depth_50)...,"0,6340579710144928","0,7460011217897268","0,5468351553270535","0,5946873820672923"
7,Bogor_XGBoost_(n_estimators_100)(max_depth_100...,"0,6340579710144928","0,7460011217897268","0,5468351553270535","0,5946873820672923"
8,Bogor_XGBoost_(n_estimators_250)(max_depth_50)...,"0,6327639751552795","0,7450222929855657","0,5461905999250499","0,5941429161979541"
9,Bogor_XGBoost_(n_estimators_250)(max_depth_100...,"0,6327639751552795","0,7450222929855657","0,5461905999250499","0,5941429161979541"


In [36]:
get_hpt_result(city, 'XGBoost', 'test')

,model_conf,accuracy,precision,recall,f1-score
0,Bogor_XGBoost_(n_estimators_500)(max_depth_10)...,"0,6282240235814296","0,7459653379271122","0,5469557625286073","0,5931845048109875"
1,Bogor_XGBoost_(n_estimators_250)(max_depth_10)...,"0,6252763448784082","0,743702200226755","0,5449402249574807","0,5912280050369012"
2,Bogor_XGBoost_(n_estimators_100)(max_depth_10)...,"0,619933677229182","0,7394408578888362","0,541905442230279","0,5889596501972428"
3,Bogor_XGBoost_(n_estimators_100)(max_depth_10)...,"0,6088798820928519","0,7302904018638433","0,5367184813955453","0,5855501314665875"
4,Bogor_XGBoost_(n_estimators_250)(max_depth_50)...,"0,619933677229182","0,7407212475633528","0,5402291748624062","0,5853576674408673"
5,Bogor_XGBoost_(n_estimators_250)(max_depth_100...,"0,619933677229182","0,7407212475633528","0,5402291748624062","0,5853576674408673"
6,Bogor_XGBoost_(n_estimators_100)(max_depth_50)...,"0,6110906411201179","0,7324871656376052","0,5356952912371068","0,5827265662030446"
7,Bogor_XGBoost_(n_estimators_100)(max_depth_100...,"0,6110906411201179","0,7324871656376052","0,5356952912371068","0,5827265662030446"
8,Bogor_XGBoost_(n_estimators_250)(max_depth_50)...,"0,6066691230655858","0,729235047607753","0,5326924271653909","0,5798406053579247"
9,Bogor_XGBoost_(n_estimators_250)(max_depth_100...,"0,6066691230655858","0,729235047607753","0,5326924271653909","0,5798406053579247"
